# 01 - Ray 基础

## 学习目标

- 了解 Ray 是什么以及它的核心抽象
- 掌握 Ray Tasks 的使用方法
- 掌握 Ray Actors 的使用方法
- 理解 Object Store 的工作原理
- 学会资源管理和错误处理

## 1. 什么是 Ray？

Ray 是一个**通用分布式计算框架**，可以将 Python 代码轻松扩展到多核和多机环境。

### 核心抽象

| 抽象 | 说明 | 适用场景 |
|------|------|----------|
| **Tasks** | 无状态的远程函数 | 并行计算、批处理 |
| **Actors** | 有状态的远程类 | 有状态服务、计数器 |
| **Objects** | 分布式共享对象 | 大数据传递、中间结果共享 |

### Ray 与 Daft 的关系

Daft 可以使用 Ray 作为执行后端，将 DataFrame 操作分布到 Ray 集群上执行：

```
+------------------+
|   Daft DataFrame |
|   (Query Plan)   |
+--------+---------+
         |
         v
+--------+---------+
|   Ray Runner     |
|   (Scheduler)    |
+--------+---------+
         |
    +----+----+
    |    |    |
    v    v    v
+--+ +--+ +--+
|W1| |W2| |W3|
+--+ +--+ +--+
  Ray Workers
```

本 Notebook 先学习 Ray 本身，下一个 Notebook 再学习 Daft + Ray 集成。

## 2. 初始化 Ray

In [ ]:
import ray
import time

print(f"Ray 版本: {ray.__version__}")

In [ ]:
# 初始化本地 Ray 集群
# address=None 表示启动本地单机集群
ray.init()

print(f"Ray 已初始化: {ray.is_initialized()}")
print(f"Dashboard 地址: http://127.0.0.1:8265")

In [ ]:
# 查看集群资源
resources = ray.cluster_resources()
print("集群资源:")
for key, value in sorted(resources.items()):
    print(f"  {key}: {value}")

## 3. Ray Tasks

Ray Task 是最基本的并行单元。通过 `@ray.remote` 装饰器将普通函数转为远程函数。

### 3.1 基础用法

In [ ]:
# 定义一个远程函数
@ray.remote
def square(x):
    """计算平方"""
    return x * x

# 调用远程函数 —— 返回 ObjectRef，不是结果
future = square.remote(4)
print(f"返回类型: {type(future)}")
print(f"ObjectRef: {future}")

# 获取结果
result = ray.get(future)
print(f"结果: {result}")

### 3.2 串行 vs 并行对比

In [ ]:
@ray.remote
def slow_square(x):
    """模拟耗时计算"""
    time.sleep(1)
    return x * x

data = list(range(8))

# 串行执行
start = time.time()
serial_results = []
for x in data:
    serial_results.append(ray.get(slow_square.remote(x)))
serial_time = time.time() - start
print(f"串行执行: {serial_time:.2f}s, 结果: {serial_results}")

# 并行执行
start = time.time()
futures = [slow_square.remote(x) for x in data]
parallel_results = ray.get(futures)
parallel_time = time.time() - start
print(f"并行执行: {parallel_time:.2f}s, 结果: {parallel_results}")

print(f"\n加速比: {serial_time / parallel_time:.1f}x")

### 3.3 Task 依赖链

可以将一个 Task 的 ObjectRef 直接传给另一个 Task，Ray 会自动处理依赖关系。

In [ ]:
@ray.remote
def add(x, y):
    return x + y

@ray.remote
def multiply(x, y):
    return x * y

# 构建依赖链: (2 + 3) * (4 + 5)
sum1 = add.remote(2, 3)       # 并行执行
sum2 = add.remote(4, 5)       # 并行执行
product = multiply.remote(sum1, sum2)  # 等待 sum1, sum2 完成后执行

result = ray.get(product)
print(f"(2 + 3) * (4 + 5) = {result}")

### 3.4 实际场景：并行文本处理

In [ ]:
@ray.remote
def process_text(text):
    """模拟文本处理：统计词频"""
    words = text.lower().split()
    word_count = {}
    for word in words:
        word = word.strip('.,!?;:')
        if word:
            word_count[word] = word_count.get(word, 0) + 1
    return word_count

def merge_word_counts(counts_list):
    """合并多个词频字典"""
    merged = {}
    for counts in counts_list:
        for word, count in counts.items():
            merged[word] = merged.get(word, 0) + count
    return merged

# 模拟多段文本
texts = [
    "Ray is a distributed computing framework for Python applications",
    "Ray makes it easy to scale Python code from laptop to cluster",
    "Daft uses Ray as a distributed execution backend for DataFrames",
    "Python is great for data processing and Ray makes it distributed",
]

# 并行处理每段文本
count_futures = [process_text.remote(text) for text in texts]

# 获取所有结果并合并
counts = ray.get(count_futures)
result = merge_word_counts(counts)

# 按词频排序输出 Top 10
sorted_words = sorted(result.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 词频:")
for word, count in sorted_words:
    print(f"  {word}: {count}")

## 4. Ray Actors

Actor 是有状态的远程对象。适用于需要维护状态的场景，如计数器、缓存、模型服务等。

### 4.1 基础 Actor

In [ ]:
@ray.remote
class Counter:
    """分布式计数器"""
    def __init__(self):
        self.count = 0

    def increment(self):
        self.count += 1
        return self.count

    def get_count(self):
        return self.count

# 创建 Actor 实例
counter = Counter.remote()

# 调用 Actor 方法
for _ in range(5):
    counter.increment.remote()

# 获取最终状态
result = ray.get(counter.get_count.remote())
print(f"计数器值: {result}")

### 4.2 多 Actor 并行

In [ ]:
@ray.remote
class DataProcessor:
    """有状态的数据处理器"""
    def __init__(self, name):
        self.name = name
        self.processed_count = 0
        self.results = []

    def process(self, items):
        """处理一批数据"""
        for item in items:
            self.results.append(item * 2)
            self.processed_count += 1
        return f"{self.name} 处理了 {len(items)} 条数据"

    def get_stats(self):
        return {
            "name": self.name,
            "processed_count": self.processed_count,
            "sample_results": self.results[:5]
        }

# 创建多个处理器
processors = [DataProcessor.remote(f"Worker-{i}") for i in range(3)]

# 分配数据到不同处理器
all_data = list(range(30))
chunk_size = len(all_data) // len(processors)

futures = []
for i, processor in enumerate(processors):
    start_idx = i * chunk_size
    end_idx = start_idx + chunk_size if i < len(processors) - 1 else len(all_data)
    chunk = all_data[start_idx:end_idx]
    futures.append(processor.process.remote(chunk))

# 等待所有处理完成
messages = ray.get(futures)
for msg in messages:
    print(msg)

# 查看各处理器统计
print("\n各处理器统计:")
stats = ray.get([p.get_stats.remote() for p in processors])
for s in stats:
    print(f"  {s['name']}: 处理 {s['processed_count']} 条, 示例: {s['sample_results']}")

## 5. Object Store

Ray Object Store 是一个分布式共享内存系统，用于在 Tasks 和 Actors 之间高效传递数据。

In [ ]:
import numpy as np

# 将大对象放入 Object Store
large_array = np.random.rand(1000000)
ref = ray.put(large_array)
print(f"ObjectRef: {ref}")
print(f"原始数组大小: {large_array.nbytes / 1024 / 1024:.2f} MB")

In [ ]:
@ray.remote
def compute_stats(array_ref):
    """计算数组统计信息（零拷贝读取）"""
    return {
        "mean": float(np.mean(array_ref)),
        "std": float(np.std(array_ref)),
        "min": float(np.min(array_ref)),
        "max": float(np.max(array_ref)),
    }

@ray.remote
def compute_percentiles(array_ref):
    """计算分位数"""
    return {
        "p25": float(np.percentile(array_ref, 25)),
        "p50": float(np.percentile(array_ref, 50)),
        "p75": float(np.percentile(array_ref, 75)),
    }

# 多个 Task 共享同一个对象（零拷贝）
stats_future = compute_stats.remote(ref)
percentiles_future = compute_percentiles.remote(ref)

stats, percentiles = ray.get([stats_future, percentiles_future])
print("统计信息:", stats)
print("分位数:", percentiles)

## 6. 资源管理

Ray 允许为 Task 和 Actor 指定资源需求，调度器会据此分配资源。

In [ ]:
# 查看可用资源
print("集群总资源:")
for k, v in sorted(ray.cluster_resources().items()):
    print(f"  {k}: {v}")

print("\n可用资源:")
for k, v in sorted(ray.available_resources().items()):
    print(f"  {k}: {v}")

In [ ]:
# 指定 Task 资源需求
@ray.remote(num_cpus=2)
def heavy_computation(n):
    """需要 2 个 CPU 的计算任务"""
    total = sum(i * i for i in range(n))
    return total

result = ray.get(heavy_computation.remote(1000000))
print(f"计算结果: {result}")

# 指定 Actor 资源需求
@ray.remote(num_cpus=1)
class LightService:
    def ping(self):
        return "pong"

service = LightService.remote()
print(f"服务响应: {ray.get(service.ping.remote())}")

## 7. 错误处理

Ray Task 中的异常会在 `ray.get()` 时重新抛出。

In [ ]:
@ray.remote
def risky_task(x):
    """可能失败的任务"""
    if x < 0:
        raise ValueError(f"输入值不能为负数: {x}")
    return x ** 0.5

# 正常调用
result = ray.get(risky_task.remote(16))
print(f"sqrt(16) = {result}")

# 异常捕获
try:
    result = ray.get(risky_task.remote(-1))
except ray.exceptions.RayTaskError as e:
    print(f"捕获到远程异常: {e}")

In [ ]:
# 超时设置
@ray.remote
def slow_task():
    time.sleep(10)
    return "完成"

try:
    result = ray.get(slow_task.remote(), timeout=2)
except ray.exceptions.GetTimeoutError:
    print("任务超时！2 秒内未完成。")

## 总结

| 概念 | 说明 | 关键 API |
|------|------|----------|
| 初始化 | 启动本地/远程 Ray 集群 | `ray.init()` |
| Tasks | 无状态远程函数 | `@ray.remote` + `.remote()` |
| Actors | 有状态远程类 | `@ray.remote class` |
| Object Store | 分布式共享内存 | `ray.put()` / `ray.get()` |
| 资源管理 | 指定 CPU/GPU 需求 | `@ray.remote(num_cpus=N)` |
| 错误处理 | 异常捕获和超时 | `try/except` + `timeout` |

## 练习

1. **并行求和**：将一个包含 100 个元素的列表分成 4 份，用 4 个 Ray Task 并行求和，再合并结果。
2. **Actor 缓存**：实现一个 `Cache` Actor，支持 `put(key, value)` 和 `get(key)` 方法。
3. **容错处理**：编写一个会随机失败的 Task（50% 概率），用循环重试直到成功。

## 下一步

继续学习 [02_daft_on_ray.ipynb](./02_daft_on_ray.ipynb) —— 将 Daft 与 Ray 结合，实现分布式数据处理。

In [ ]:
# 清理资源
ray.shutdown()
print("Ray 已关闭")